### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [9]:
import pandas as pd
import urllib.request
import json 
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

def get_prediction(field_array=None, myurl=None):
    """
    Input parameters:
        field_array: python list - contains raw fields needed to make a prediction
        myurl: REST API url
    Returns:
        result: float - model prediction
    """
    result = 0
    description, company_profile, benefits = field_array
    body = {'description': description, 
            'company_profile': company_profile,
            'benefits': benefits} 

    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    
    return json.loads(response.read())['predictions']

API_URL = "http://0.0.0.0:8180/predict"

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Посмотрим что за данные у нас есть

In [10]:
X_test[['description', 'company_profile', 'benefits']].head(3)

,description,company_profile,benefits
0,Stylect is a dynamic startup that helps helps ...,NaN,We are negotiable on salary and there is the p...
1,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,NaN


### Getting predictions from REST API

In [13]:
%%time
predictions = X_test[['description', 'company_profile', 
                      'benefits']].iloc[:500].apply(lambda x: get_prediction(x, API_URL), 1)

CPU times: user 323 ms, sys: 58.8 ms, total: 382 ms
Wall time: 5.23 s


check performance (roc auc)

In [14]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

Давайте подумаем, где наше api может сломаться

In [15]:
get_prediction(field_array=("", None, None), myurl=API_URL)

0.13518841848029955

In [16]:
get_prediction(field_array=("", "", ""), myurl=API_URL)

0.13518841848029955